In [ ]:
import torch
import numpy as np
import torch.nn as nn
from scipy.io import loadmat
import matplotlib.pyplot as plt
from Dataset import Dataset, ELM, device
torch.set_default_dtype(torch.float64)

In [ ]:
train = loadmat('data_training_reactor_pol.mat')
test = loadmat('data_validation_reactor_pol.mat')

y_train = 0.0001*(train['NAMW'][0]-train['NAMW'][0][0])
y_test = 0.0001*(test['NAMW'][0]-test['NAMW'][0][0])

x_train = np.concatenate((100*np.concatenate(([[0]], (train['FI'].T[1:]-train['FI'].T[0])), axis=0),
                          100*np.concatenate(([[0], [0]], (train['FI'].T[2:]-train['FI'].T[0])), axis=0),
                          np.concatenate(([0], y_train[1:]), axis=0).reshape(-1, 1),
                          np.concatenate(([0, 0], y_train[2:]), axis=0).reshape(-1, 1)), axis=1)

x_test = np.concatenate((100*np.concatenate(([[0]], (test['FI'].T[1:]-test['FI'].T[0])), axis=0),
                         100*np.concatenate(([[0], [0]], (test['FI'].T[2:]-test['FI'].T[0])), axis=0),
                         np.concatenate(([0], y_test[1:]), axis=0).reshape(-1, 1),
                         np.concatenate(([0, 0], y_test[2:]), axis=0).reshape(-1, 1)), axis=1)

In [ ]:
trainset = Dataset(x_train, y_train)
testset = Dataset(x_test, y_test)

activation = nn.Sigmoid()

In [ ]:
model = ELM(4, 50, 1).to(device)

V = activation(model.fc1.bias.data.reshape(-1, 1) + model.fc1.weight.data @ trainset[:][0].T.to(device))
H = torch.concatenate([torch.ones(1, len(trainset)).to(device), V.to(device)], dim=0).T
T = trainset[:][1].reshape(1, -1).T.to(device)
W2 = torch.linalg.pinv(H) @ T.reshape(1, -1, 1)
model.fc2.bias.data, model.fc2.weight.data = W2[0][0], W2[0][1:].T

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
max_train_error = []
min_train_error = []
max_test_error = []
min_test_error = []
num_neurons = 200
for j in range(1, num_neurons+1):
    train_error = []
    test_error = []
    for i in range(10):
        model = ELM(4, 5*j, 1).to(device)
        V = activation(model.fc1.bias.data.reshape(-1, 1) + model.fc1.weight.data @ trainset[:][0].T.to(device))
        H = torch.concatenate([torch.ones(1, len(trainset)).to(device), V.to(device)], dim=0).T
        T = trainset[:][1].reshape(1, -1).T.to(device)
        W2 = torch.linalg.pinv(H) @ T.reshape(1, -1, 1) # bez regularyzacji
        # P = torch.linalg.inv(H.T @ H + 1e-3*torch.eye(H.shape[1]).to(device))
        # W2 = P @ H.T @ T.reshape(1, -1, 1)
        model.fc2.bias.data, model.fc2.weight.data = W2[0][0], W2[0][1:].T
        train_error.append((model(trainset[:][0].clone().detach().reshape(-1, 4).to(device))-trainset[:][1].reshape(-1, 1).to(device)).reshape(-1).pow(2).mean())
        test_error.append((model(testset[:][0].clone().detach().reshape(-1, 4).to(device))-testset[:][1].reshape(-1, 1).to(device)).reshape(-1).pow(2).mean())
    max_train_error.append(max(train_error).item())
    min_train_error.append(min(train_error).item())
    max_test_error.append(max(test_error).item())
    min_test_error.append(min(test_error).item())
ax[0].plot([5*i+5 for i in range(num_neurons)], max_train_error, 'r', label='Błąd maksymalny')
ax[0].plot([5*i+5 for i in range(num_neurons)], min_train_error, 'b', label='Błąd minimalny')
ax[0].set_ylabel('Błąd średniokwadratowy zbioru uczącego')
ax[0].set_xlabel('Liczba neuronów')
ax[0].set_yscale('log')
ax[0].grid()
ax[0].legend(loc='upper right')
ax[1].plot([5*i+5 for i in range(num_neurons)], max_test_error, 'r', label='Błąd maksymalny')
ax[1].plot([5*i+5 for i in range(num_neurons)], min_test_error, 'b', label='Błąd minimalny')
ax[1].set_ylabel('Błąd średniokwadratowy zbioru weryfikującego')
ax[1].set_xlabel('Liczba neuronów')
ax[1].set_yscale('log')
ax[1].grid()
ax[1].legend(loc='upper right')
plt.show()
from decimal import Decimal
for i, error in enumerate(min_test_error):
    if i+1 in [1, 2, 10, 20, 100, 200]:
        print(f'{Decimal(error):.2e}', end=' & ')

In [ ]:
print(f'Train error: {(model(trainset[:][0].clone().detach().reshape(-1, 4).to(device))-trainset[:][1].reshape(-1, 1).to(device)).reshape(-1).pow(2).mean()}')
print(f'Test error: {(model(testset[:][0].clone().detach().reshape(-1, 4).to(device))-testset[:][1].reshape(-1, 1).to(device)).reshape(-1).pow(2).mean()}')
plt.plot(y_test)
plt.plot(model(torch.tensor(x_test).reshape(-1, 4).to(device)).cpu().detach().numpy(), 'r--')
plt.legend(['Dane weryfikujące', 'Model'], loc='upper right')
plt.xlabel('x')
plt.ylabel('y')
plt.grid()
plt.show()

In [ ]:
plt.plot([np.min(y_test), np.max(y_test)], [np.min(y_test), np.max(y_test)])
plt.plot(y_test, model(torch.tensor(x_test).reshape(-1, 4).to(device)).cpu().detach().numpy(), '.', markersize=1)
plt.grid()
plt.show()